## Let's create a golden test set with Synthetic Data Generation!

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import JSONLoader
import json

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["title"] = record.get("title", "")
    metadata["tags"] = record.get("tags", [])
    return metadata

loader = DirectoryLoader(
    path="data/designs",
    glob="**/*.json",
    loader_cls=JSONLoader,
    loader_kwargs={
        "jq_schema": ".",  # This will select the entire JSON object
        "content_key": None,  # We're not selecting a specific content key
        "metadata_func": metadata_func,
        "text_content": False
    }
)

documents = loader.load()

print(f"Loaded {len(documents)} documents")
if documents:
    print("\nExample document:")
    print(f"Content: {documents[0].page_content}")
    print(f"Metadata: {documents[0].metadata}")


Loaded 141 documents

Example document:
Content: {"id": "135", "url": "https://www.csszengarden.com/135", "css_url": "https://www.csszengarden.com/135/135.css", "description": "This design uses an elegant and traditional aesthetic, with a muted color palette featuring cream and terracotta tones. The use of a patterned background adds texture, while the vertical layout and serif typography provide a classic and sophisticated feel. The design balances text-heavy content with organized sections, making information easily accessible, and its ornate flourishes add a touch of refinement.", "categories": ["Traditional", "Elegant", "Text-Heavy", "Classic"], "visual_characteristics": ["Muted Color Palette", "Vertical Layout", "Serif Typography", "Textured Background", "Ornate Flourishes"]}
Metadata: {'source': '/Users/owner/Desktop/Projects/ai-maker-space/code/ImagineUI/src/data/designs/135/metadata.json', 'seq_num': 1, 'title': '', 'tags': []}


Now let's create a knowledge graph based on our design metadata.

In [13]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# using 4o-mini due to rate limits
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [14]:
from ragas.testset.graph import KnowledgeGraph, Node, NodeType

kg = KnowledgeGraph()

for doc in documents:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

kg

KnowledgeGraph(nodes: 141, relationships: 0)

In [15]:
from ragas.testset.transforms import default_transforms, apply_transforms

# why use these new variables?
transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=documents, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/141 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/141 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/423 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 141, relationships: 795)

In [17]:
kg.save("css_zen_garden_design_data")
design_kg = KnowledgeGraph.load("css_zen_garden_design_data")

Okay! With out knowledge graph of relationships, we can now generate a test set.

In [18]:
from ragas.testset import TestsetGenerator

testset_generator = TestsetGenerator(
    knowledge_graph=design_kg,
    llm=generator_llm,
    embedding_model=generator_embeddings
)


In [ ]:
from ragas.testset.synthesizers import default_query_distribution, 

query_distribution = default_query_distribution(
    kg=design_kg,
    llm=generator_llm
)

testset = testset_generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How do ornate flourishes contribute to the ove...,"[{""id"": ""135"", ""url"": ""https://www.csszengarde...",Ornate flourishes in web design add a touch of...,single_hop_specifc_query_synthesizer
1,How does the nature-inspired design in the pro...,"[{""id"": ""132"", ""url"": ""https://www.csszengarde...",The nature-inspired design features a minimali...,single_hop_specifc_query_synthesizer
2,Can you describe the design principles and vis...,"[{""id"": ""104"", ""url"": ""https://www.csszengarde...",The CSS Zen Garden design emphasizes a harmoni...,single_hop_specifc_query_synthesizer
3,How does the use of burgundy in web design con...,"[{""id"": ""103"", ""url"": ""https://www.csszengarde...",The design features a dark burgundy background...,single_hop_specifc_query_synthesizer
4,How vintage aesthetic show in this design?,"[{""id"": ""168"", ""url"": ""https://www.csszengarde...",The design cleverly combines a vintage aesthet...,single_hop_specifc_query_synthesizer
5,How do the designs at CSS Zen Garden utilize a...,"[<1-hop>\n\n{""id"": ""151"", ""url"": ""https://www....",The designs at CSS Zen Garden utilize a light ...,multi_hop_specific_query_synthesizer
6,How do the typography-focused designs in the d...,"[<1-hop>\n\n{""id"": ""155"", ""url"": ""https://www....",The typography-focused design in the dark them...,multi_hop_specific_query_synthesizer
7,What are the illustrative elements used in the...,"[<1-hop>\n\n{""id"": ""218"", ""url"": ""https://www....",The illustrative elements used in the designs ...,multi_hop_specific_query_synthesizer
8,What are the key visual characteristics of the...,"[<1-hop>\n\n{""id"": ""208"", ""url"": ""https://www....",The design at https://www.csszengarden.com/208...,multi_hop_specific_query_synthesizer
9,Wht are the key visual characteristics of the ...,"[<1-hop>\n\n{""id"": ""001"", ""url"": ""https://www....",The key visual characteristics of the Zen Gard...,multi_hop_specific_query_synthesizer


Just for comparison, let's see what the simplified version with langchain docs looks like.



In [22]:
generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings
)

testset = generator.generate_with_langchain_docs(
    documents=documents,
    testset_size=10,
    query_distribution=query_distribution
)

testset.to_pandas()




Applying SummaryExtractor:   0%|          | 0/141 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/141 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/423 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Can you tell me more about what makes this des...,"[{""id"": ""135"", ""url"": ""https://www.csszengarde...",This design is classic because it employs an e...,single_hop_specifc_query_synthesizer
1,What makes this design typography-focused?,"[{""id"": ""132"", ""url"": ""https://www.csszengarde...",This design is typography-focused because it u...,single_hop_specifc_query_synthesizer
2,How does the web design concept illustrated in...,"[{""id"": ""104"", ""url"": ""https://www.csszengarde...",The web design concept illustrated in the CSS ...,single_hop_specifc_query_synthesizer
3,What make this design vintage?,"[{""id"": ""103"", ""url"": ""https://www.csszengarde...",This design features a vintage theme through i...,single_hop_specifc_query_synthesizer
4,How does the vintage aesthetic influence the d...,"[{""id"": ""168"", ""url"": ""https://www.csszengarde...",The vintage aesthetic in the design is cleverl...,single_hop_specifc_query_synthesizer
5,What are the key visual characteristics that m...,"[<1-hop>\n\n{""id"": ""201"", ""url"": ""https://www....",The design at CSS Zen Garden employs a strong ...,multi_hop_specific_query_synthesizer
6,What are the visual characteristics of designs...,"[<1-hop>\n\n{""id"": ""008"", ""url"": ""https://www....",The designs that effectively utilize bold typo...,multi_hop_specific_query_synthesizer
7,What are the key visual characteristics of the...,"[<1-hop>\n\n{""id"": ""151"", ""url"": ""https://www....",The key visual characteristics of the minimali...,multi_hop_specific_query_synthesizer
8,How do the subtle background colors and subtle...,"[<1-hop>\n\n{""id"": ""164"", ""url"": ""https://www....",The design featuring a minimalist layout with ...,multi_hop_specific_query_synthesizer
9,In what ways do the designs from CSS Zen Garde...,"[<1-hop>\n\n{""id"": ""109"", ""url"": ""https://www....",The designs from CSS Zen Garden exemplify info...,multi_hop_specific_query_synthesizer
